In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing Dependencies**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras.applications as app
import keras.utils as utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as KB
import cv2

In [ ]:
img_size=224

**make_data** converts the images into the numpy arrays.
<br>
**pathc** : path of the directory where cat images are present
<br>
**pathd** : path of the directory where dog images are present

In [ ]:
def make_data(pathc=None,pathd=None,size=(224,224)):
    data=[]
    y=[]
    j=0
    if len(os.listdir(pathd))!=0:
        for i in os.listdir(pathd):
            if i[-3:] == 'jpg':
                data_dogs=cv2.imread(pathd+'/'+str(i))
                res=cv2.resize(data_dogs,size) 
                y.append(0)
                data.append(res)
    if len(os.listdir(pathc))!=0:
        for i in os.listdir(pathc):
            if i[-3:] == 'jpg':
                data_cats=cv2.imread(pathc+'/'+str(i))
                res=cv2.resize(data_cats,size) 
                y.append(1)
                data.append(res)
    y=np.array(y)
    data=np.array(data)
    return data,y

In [ ]:
X_train,y_train=make_data(pathd='/kaggle/input/cat-and-dog/training_set/training_set/dogs',
                          pathc='/kaggle/input/cat-and-dog/training_set/training_set/cats',)
# os.listdir('../input/cat-and-dog/trainin')

In [ ]:
X_train.shape,y_train.shape

<h1>Converting y_train categorical labels</h1>

In [ ]:
y_cate_train=utils.to_categorical(y_train)

In [ ]:
y_cate_train.shape

In [ ]:
plt.subplot(1,2,1)
plt.imshow(X_train[0])
plt.title('dog -> '+str(y_cate_train[0]))

plt.subplot(1,2,2)
plt.imshow(X_train[-1])
plt.title('cat -> '+str(y_cate_train[-1]))


<h2>Dog category is represented as [1 , 0]<br>
    Cat category is represented as  [0 , 1]</h2>


In [ ]:
def clear():
    KB.clear_session()    

In [ ]:
clear()

In [ ]:
from keras.layers import Dense,MaxPooling2D,GlobalAveragePooling2D

In [ ]:
base_model=app.ResNet50(include_top=False,input_shape=(img_size,img_size,3),weights='imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
x=base_model.output
pool=GlobalAveragePooling2D()(x)

In [ ]:
output=Dense(2,activation='softmax')(pool)

In [ ]:
from keras.models import Model
model=Model(inputs=base_model.input,outputs=output)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
y_cate_train.shape,X_train.shape

In [ ]:
model

In [ ]:
model.fit(X_train,y_cate_train,epochs=5)

In [ ]:
X_test,y_test=make_data(pathd='/kaggle/input/cat-and-dog/test_set/test_set/dogs',
                          pathc='/kaggle/input/cat-and-dog/test_set/test_set/cats',)
# os.listdir('../input/cat-and-dog/trainin')

In [ ]:
y_cate_test=utils.to_categorical(y_test)

In [ ]:
model.evaluate(X_test,y_cate_test)

In [ ]:
model.save('cat-dog-ResNet50.h5')